In [2]:
from sklearn import cross_validation,model_selection
from sklearn import svm  
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn import cross_validation,model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from sklearn.feature_extraction.text import TfidfTransformer
from string import count
ps = PorterStemmer()

/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/home/umang/.local/lib/python2.7/site-packages

/home/umang/.local/lib/python2.7/site-packages/pandas/core/window.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.window as _window
/home/umang/.local/lib/python2.7/site-packages/pandas/core/groupby/groupby.py:68: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (lib, reduction,
/home/umang/.local/lib/python2.7/site-packages/pandas/core/reshape/reshape.py:30: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos as _algos, reshape as _reshape
/home/umang/.local/lib/python2.7/site-packages/pandas/io/parsers.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.parsers as parsers
/home/umang/.local/lib/python2.7/site-packages/pandas/io/pytables.py:50: RuntimeWarning: numpy.dtype size 

In [3]:
word2vec_features = pd.read_csv("../AM_Email_Tagging/Intermediate_Excel_Files/ummang_word2vec.csv")


/home/umang/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
word2vec_features.head()

,Unnamed: 0,Amcatid,Sentences,Tag_name,Flow,Rater_id,All_tags,cleaned_text,0,1,...,292,293,294,295,296,297,298,299,flag,Unnamed: 309
0,0,10016611374320,"I am Rajeev Pathak, CSE,IT Systems",oi_tag,3,1,"['oi', '/oi', 'ccd', '/ccd']",i am rajeev pathak cse it systems,0.00341797,-0.187500,...,0.002563,0.336914,0.118469,-0.143799,0.098267,-0.203857,-0.007050,0.055481,0.0,NaN
1,1,10016611374320,. I can send you the detailed time and cost es...,ccd_tag,3,1,"['oi', '/oi', 'ccd', '/ccd']",i can send you the detailed time and cost es...,-0.0317383,-0.119141,...,-0.262939,-0.197815,0.301208,0.294434,0.207520,-0.430069,0.114990,-0.767578,0.0,NaN
2,2,10016614250601,Super Bazaar is looking to upgrade its old in...,op_tag,1,1,"['op', '/op', 'cdf', '/cdf']",super bazaar is looking to upgrade its old in...,0.49353,0.235840,...,0.121674,0.478882,0.555786,-0.464111,-0.050537,-0.084595,0.132950,-0.100220,0.0,NaN
3,3,10016614250601,we will tell you our simpler way to deal.\n\n\n,cdf_tag,1,1,"['op', '/op', 'cdf', '/cdf']",we will tell you our simpler way to deal,0.247253,-0.041504,...,-0.165283,-0.085449,-0.573730,0.235352,0.309509,-0.439819,-0.047852,0.007416,0.0,NaN
4,4,10016616206567,This is to inform you sir that we offer the mo...,op_tag,2,1,"['op', '/op', 'ccd', '/ccd']",this is to inform you sir that we offer the mo...,-0.264114,-0.421753,...,-0.791138,0.642151,0.433533,-0.424728,0.186951,-0.890274,0.104950,-0.127930,0.0,NaN


In [69]:
all_sentences_stemmed = []
all_tags = []
opening_tags = ['op_tag','oi_tag','opc_tag','ow_tag','oa_tag','ocg_tag','odf_tag']
closing_tags = ['cac_tag','cap_tag','ccd_tag','cdf_tag','cff_tag','cth_tag','cw_tag']
i = 0
for row in range(0,word2vec_features.index.size):
    if word2vec_features['Tag_name'][i] in opening_tags or word2vec_features['Tag_name'][i] in closing_tags:
        all_tags.append(word2vec_features['Tag_name'][i])
        splitted_arr = word2vec_features['cleaned_text'][i].split(" ")
        sentence= []
        for word in splitted_arr:
            sentence.append(ps.stem(word))
        sentence = " ".join(sentence)
        word2vec_features = word2vec_features.set_value(row,'Cleaned Stemmed Text', sentence)
#         all_sentences_stemmed.append(sentence)
    i += 1


/home/umang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [183]:
word2vec = word2vec_features
word2vec.head()


,Unnamed: 0,Amcatid,Sentences,Tag_name,Flow,Rater_id,All_tags,cleaned_text,0,1,...,293,294,295,296,297,298,299,flag,Unnamed: 309,Cleaned Stemmed Text
0,0,10016611374320,"I am Rajeev Pathak, CSE,IT Systems",oi_tag,3,1,"['oi', '/oi', 'ccd', '/ccd']",i am rajeev pathak cse it systems,0.00341797,-0.187500,...,0.336914,0.118469,-0.143799,0.098267,-0.203857,-0.007050,0.055481,0.0,,i am rajeev pathak cse it system
1,1,10016611374320,. I can send you the detailed time and cost es...,ccd_tag,3,1,"['oi', '/oi', 'ccd', '/ccd']",i can send you the detailed time and cost es...,-0.0317383,-0.119141,...,-0.197815,0.301208,0.294434,0.207520,-0.430069,0.114990,-0.767578,0.0,,i can send you the detail time and cost esti...
2,2,10016614250601,Super Bazaar is looking to upgrade its old in...,op_tag,1,1,"['op', '/op', 'cdf', '/cdf']",super bazaar is looking to upgrade its old in...,0.49353,0.235840,...,0.478882,0.555786,-0.464111,-0.050537,-0.084595,0.132950,-0.100220,0.0,,super bazaar is look to upgrad it old invento...
3,3,10016614250601,we will tell you our simpler way to deal.\n\n\n,cdf_tag,1,1,"['op', '/op', 'cdf', '/cdf']",we will tell you our simpler way to deal,0.247253,-0.041504,...,-0.085449,-0.573730,0.235352,0.309509,-0.439819,-0.047852,0.007416,0.0,,we will tell you our simpler way to deal
4,4,10016616206567,This is to inform you sir that we offer the mo...,op_tag,2,1,"['op', '/op', 'ccd', '/ccd']",this is to inform you sir that we offer the mo...,-0.264114,-0.421753,...,0.642151,0.433533,-0.424728,0.186951,-0.890274,0.104950,-0.127930,0.0,,thi is to inform you sir that we offer the mod...


In [9]:
# all_opening_sentences
all_sentences_clean = []
for sentence in all_sentences_stemmed:
     all_sentences_clean.append(" ".join(sentence))
all_sentences_clean

[u'i am rajeev pathak  cse it system',
 u'  i can send you the detail time and cost estim if you are interest ',
 u' super bazaar is look to upgrad it old inventori manag comput system ',
 'we will tell you our simpler way to deal     ',
 u'thi is to inform you sir that we offer the modern comput system with rfid track and real time cost report',
 u'we will tri our best to serv you     ',
 u'superbazaar upgrad the old inventori comput system in the mark',
 u' i wish our compani fulfil your need     ',
 u' thi mail is regard your requir about new technolog inventori system and rifd tracker',
 u'pleas respond with all the requir detail ',
 u' i hope to get a speedi repli from your end     ',
 u'thi is withrefer to your letter demand suitabl propos about the new and modern inventori system with rfid track and real time cost report ',
 'we are leader in it sector ',
 u'hope you will give us a chanc to serv you',
 u'i am rajeev pathak as a corpor sale execut in it system',
 u'it honour that

In [10]:
print(len(word2vec_features['cleaned_text']))
print(len(all_sentences_clean))

9910
9903


In [12]:
db = pd.read_csv("../AM_Email_Tagging/All_Tags_Keywords.csv")
db.head()

,op_tag,oi_tag,cdf_tag,cw_tag,cap_tag,cac_tag,ccd_tag,cth_tag,cff_tag,odf_tag,ocg_tag,oa_tag,ow_tag,opc_tag
0,inform,Hello,know,thank,sorry,respond,queries,thankful,response,inform,happy,complaint,warm,response
1,regarding,name,request,day,regretful,send,provide,taking,listening,attention,glad,sorry,greetings,conversation
2,context,myself,interested,nice,inconvenience,interested,problem,obliged,waiting,received,success,deeply,pleasant,reference
3,information,Sir,assures,hope,sorry,respond,contact,choosing,response,problem,prestigious,inconvenience,regards,promised
4,attention,respect,greatfullly,contacting,apologies,send,assistance,customer,positive,good,good,guilt,NaN,days


In [13]:
i = 0
keyword_list = []
only_keyword= []
for row in db:
    for keyword in db[row]:
        if str(keyword) != 'nan':
            keyword = ps.stem(keyword.strip())
            keyword_list.append(keyword.strip())
            only_keyword.append(keyword.strip())
            
keyword_list = list(set(keyword_list))
keyword_list

[u'pleas',
 'respond',
 u'assur',
 u'regret',
 u'look',
 u'acknowledg',
 'complaint',
 u'apolog',
 'return',
 'good',
 'glad',
 u'refer',
 u'respons',
 'respect',
 'trust',
 'best',
 'sir',
 'write',
 u'convers',
 u'thank',
 u'inconveni',
 'inform',
 u'receiv',
 u'sorri',
 'send',
 u'custom',
 'note',
 u'greatfullli',
 u'take',
 u'interest',
 u'queri',
 u'deepli',
 'pride',
 'tell',
 'hope',
 'nice',
 u'happi',
 'draw',
 u'investig',
 u'insur',
 'joy',
 u'assist',
 u'regard',
 u'notic',
 'pleasant',
 u'unfortun',
 'hi',
 u'promis',
 'know',
 u'attent',
 u'prestigi',
 'day',
 'regular',
 u'wait',
 u'tri',
 'myself',
 'name',
 u'success',
 u'provid',
 'warm',
 'request',
 u'greet',
 'email',
 'matter',
 u'contact',
 'guilt',
 'context',
 u'choos',
 u'posit',
 'problem',
 u'listen',
 u'oblig',
 'hello',
 'luck']

In [16]:
import pandas as pd
bag_of_words = pd.DataFrame()
for keyword in keyword_list:
    bag_of_words.insert(0,keyword,'lol')

bag_of_words['Amcatid'] = word2vec_features['Amcatid']
bag_of_words['Cleaned Sentence'] = word2vec_features['cleaned']
bag_of_words['Flow'] = word2vec_features['Flow']
bag_of_words['Rater_id'] = word2vec_features['Rater_id']
bag_of_words['All_tags'] = word2vec_features['All_tags']
bag_of_words['Tag_name'] = word2vec_features['Tag_name']


bag_of_words

,luck,hello,oblig,listen,problem,posit,choos,context,guilt,contact,...,regret,assur,respond,pleas,Amcatid,Cleaned Sentence,Flow,Rater_id,All_tags,Tag_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016611374320,i am rajeev pathak cse it systems,3,1,"['oi', '/oi', 'ccd', '/ccd']",oi_tag
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016611374320,i can send you the detailed time and cost es...,3,1,"['oi', '/oi', 'ccd', '/ccd']",ccd_tag
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016614250601,super bazaar is looking to upgrade its old in...,1,1,"['op', '/op', 'cdf', '/cdf']",op_tag
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016614250601,we will tell you our simpler way to deal,1,1,"['op', '/op', 'cdf', '/cdf']",cdf_tag
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016616206567,this is to inform you sir that we offer the mo...,2,1,"['op', '/op', 'ccd', '/ccd']",op_tag
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016616206567,we will try our best to serve you,2,1,"['op', '/op', 'ccd', '/ccd']",ccd_tag
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016691394173,superbazaar upgrade the old inventory computer...,2,1,"['op', '/op', 'cff', '/cff']",op_tag
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016691394173,i wish our company fulfill your need,2,1,"['op', '/op', 'cff', '/cff']",cff_tag
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016691425855,this mail is regarding your requirement about...,3,1,"['op', '/op', 'cac', '/cac']",op_tag
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10016691425855,please respond with all the required details,3,1,"['op', '/op', 'cac', '/cac']",cac_tag


In [164]:
bag_of_words['hello'][0] = 'umang'
bag_of_words
X = bag_of_words['Cleaned Sentence']
print X


0                      i am rajeev pathak  cse it systems
1         i can send you the detailed time and cost es...
2        super bazaar is looking to upgrade its old in...
3           we will tell you our simpler way to deal     
4       this is to inform you sir that we offer the mo...
5                  we will try our best to serve you     
6       superbazaar upgrade the old inventory computer...
7               i wish our company fulfill your need     
8        this mail is regarding your requirement about...
9           please respond with all the required details 
10        i hope to get a speedy reply from your end     
11      this is withreference to your letter demanding...
12                            we are leader in it sector 
13            hope you will give us a chance to serve you
14      i am rajeev pathak as a corporate sales execut...
15               its honour that we are working with you 
16                it system is a leader in the it sector 
17            

/home/umang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [169]:

db = pd.read_csv('/home/umang/Desktop/AM_Email_Tagging/Intermediate Excel Files/ummang_word2vec.csv')
db.head()
array = db.values
vectors = array[:,8:308]
len(vectors)
vector_db = pd.DataFrame()
vector_db = 

TypeError: insert() takes at least 4 arguments (2 given)

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [3]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X_train)
print "Vocabulary:", count_vectorizer.vocabulary_

NameError: name 'X_train' is not defined

In [51]:
vectorizer = CountVectorizer(vocabulary = keyword_list)
vectorizer_1 = CountVectorizer(vocabulary = keyword_list)

print vectorizer_1

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary=[u'pleas', 'respond', u'assur', u'regret', u'look', u'acknowledg', 'complaint', u'apolog', 'return', 'good', 'glad', u'refer', u'respons', 'respect', 'trust', 'best', 'sir', 'write', u'convers', u'thank', u'inconveni', 'inform', u'receiv', u'sorri', 'send', u'custom', 'note', u'greatfulll...u'contact', 'guilt', 'context', u'choos', u'posit', 'problem', u'listen', u'oblig', 'hello', 'luck'])


In [87]:
word2vec_features = word2vec_features.replace(np.nan,'')
array = vectorizer_1.transform(list(word2vec_features['Cleaned Stemmed Text'])).todense()

In [89]:
array = np.array(array)

In [90]:
for i in tqdm(range(0,word2vec_features.index.size)):
    for j,keyword in enumerate(keyword_list): 
        bag_of_words[keyword][i] = array[i][j]



  0%|          | 0/9910 [00:00<?, ?it/s]/home/umang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 1/9910 [00:03<8:22:34,  3.04s/it]

  0%|          | 2/9910 [00:06<8:17:20,  3.01s/it]

  0%|          | 3/9910 [00:08<8:14:19,  2.99s/it]

  0%|          | 4/9910 [00:11<8:12:35,  2.98s/it]

  0%|          | 5/9910 [00:14<8:11:56,  2.98s/it]

  0%|          | 6/9910 [00:17<8:13:17,  2.99s/it]

  0%|          | 7/9910 [00:20<8:13:08,  2.99s/it]

  0%|          | 8/9910 [00:23<8:14:47,  3.00s/it]

  0%|          | 9/9910 [00:26<8:14:01,  2.99s/it]

  0%|          | 10/9910 [00:29<8:13:17,  2.99s/it]

  0%|          | 11/9910 [00:32<8:12:43,  2.99s/it]

  0%|         

  3%|▎         | 268/9910 [13:22<8:01:11,  2.99s/it]

  3%|▎         | 269/9910 [13:25<8:01:06,  2.99s/it]

  3%|▎         | 270/9910 [13:28<8:01:01,  2.99s/it]

  3%|▎         | 271/9910 [13:31<8:00:56,  2.99s/it]

  3%|▎         | 272/9910 [13:34<8:00:51,  2.99s/it]

  3%|▎         | 273/9910 [13:37<8:00:46,  2.99s/it]

  3%|▎         | 274/9910 [13:40<8:00:42,  2.99s/it]

  3%|▎         | 275/9910 [13:43<8:00:38,  2.99s/it]

  3%|▎         | 276/9910 [13:46<8:00:34,  2.99s/it]

  3%|▎         | 277/9910 [13:48<8:00:29,  2.99s/it]

  3%|▎         | 278/9910 [13:51<8:00:25,  2.99s/it]

  3%|▎         | 279/9910 [13:54<8:00:20,  2.99s/it]

  3%|▎         | 280/9910 [13:57<8:00:15,  2.99s/it]

  3%|▎         | 281/9910 [14:00<8:00:11,  2.99s/it]

  3%|▎         | 282/9910 [14:03<8:00:06,  2.99s/it]

  3%|▎         | 283/9910 [14:06<8:00:03,  2.99s/it]

  3%|▎         | 284/9910 [14:09<7:59:59,  2.99s/it]

  3%|▎         | 285/9910 [14:12<7:59:55,  2.99s/it]

  3%|▎         | 286/9910 [1

  5%|▌         | 540/9910 [26:41<7:43:01,  2.96s/it]

  5%|▌         | 541/9910 [26:44<7:42:58,  2.96s/it]

  5%|▌         | 542/9910 [26:46<7:42:55,  2.96s/it]

  5%|▌         | 543/9910 [26:49<7:42:51,  2.96s/it]

  5%|▌         | 544/9910 [26:52<7:42:48,  2.96s/it]

  5%|▌         | 545/9910 [26:55<7:42:44,  2.96s/it]

  6%|▌         | 546/9910 [26:58<7:42:41,  2.96s/it]

  6%|▌         | 547/9910 [27:01<7:42:37,  2.96s/it]

  6%|▌         | 548/9910 [27:04<7:42:34,  2.96s/it]

  6%|▌         | 549/9910 [27:07<7:42:30,  2.96s/it]

  6%|▌         | 550/9910 [27:10<7:42:27,  2.96s/it]

  6%|▌         | 551/9910 [27:13<7:42:23,  2.96s/it]

  6%|▌         | 552/9910 [27:16<7:42:20,  2.96s/it]

  6%|▌         | 553/9910 [27:19<7:42:17,  2.96s/it]

  6%|▌         | 554/9910 [27:22<7:42:14,  2.96s/it]

  6%|▌         | 555/9910 [27:25<7:42:10,  2.96s/it]

  6%|▌         | 556/9910 [27:28<7:42:07,  2.96s/it]

  6%|▌         | 557/9910 [27:31<7:42:03,  2.96s/it]

  6%|▌         | 558/9910 [2

  8%|▊         | 812/9910 [39:59<7:28:05,  2.96s/it]

  8%|▊         | 813/9910 [40:02<7:28:02,  2.96s/it]

  8%|▊         | 814/9910 [40:05<7:27:59,  2.96s/it]

  8%|▊         | 815/9910 [40:08<7:27:56,  2.96s/it]

  8%|▊         | 816/9910 [40:11<7:27:53,  2.96s/it]

  8%|▊         | 817/9910 [40:14<7:27:50,  2.96s/it]

  8%|▊         | 818/9910 [40:17<7:27:47,  2.96s/it]

  8%|▊         | 819/9910 [40:20<7:27:44,  2.96s/it]

  8%|▊         | 820/9910 [40:23<7:27:40,  2.95s/it]

  8%|▊         | 821/9910 [40:26<7:27:37,  2.95s/it]

  8%|▊         | 822/9910 [40:28<7:27:34,  2.95s/it]

  8%|▊         | 823/9910 [40:31<7:27:31,  2.95s/it]

  8%|▊         | 824/9910 [40:34<7:27:28,  2.95s/it]

  8%|▊         | 825/9910 [40:37<7:27:24,  2.95s/it]

  8%|▊         | 826/9910 [40:40<7:27:21,  2.95s/it]

  8%|▊         | 827/9910 [40:43<7:27:18,  2.95s/it]

  8%|▊         | 828/9910 [40:46<7:27:15,  2.95s/it]

  8%|▊         | 829/9910 [40:49<7:27:12,  2.95s/it]

  8%|▊         | 830/9910 [4

 11%|█         | 1083/9910 [53:15<7:14:05,  2.95s/it]

 11%|█         | 1084/9910 [53:18<7:14:02,  2.95s/it]

 11%|█         | 1085/9910 [53:21<7:13:59,  2.95s/it]

 11%|█         | 1086/9910 [53:24<7:13:55,  2.95s/it]

 11%|█         | 1087/9910 [53:27<7:13:52,  2.95s/it]

 11%|█         | 1088/9910 [53:30<7:13:49,  2.95s/it]

 11%|█         | 1089/9910 [53:33<7:13:46,  2.95s/it]

 11%|█         | 1090/9910 [53:36<7:13:43,  2.95s/it]

 11%|█         | 1091/9910 [53:39<7:13:40,  2.95s/it]

 11%|█         | 1092/9910 [53:41<7:13:37,  2.95s/it]

 11%|█         | 1093/9910 [53:44<7:13:34,  2.95s/it]

 11%|█         | 1094/9910 [53:47<7:13:31,  2.95s/it]

 11%|█         | 1095/9910 [53:50<7:13:28,  2.95s/it]

 11%|█         | 1096/9910 [53:53<7:13:25,  2.95s/it]

 11%|█         | 1097/9910 [53:56<7:13:22,  2.95s/it]

 11%|█         | 1098/9910 [53:59<7:13:19,  2.95s/it]

 11%|█         | 1099/9910 [54:02<7:13:16,  2.95s/it]

 11%|█         | 1100/9910 [54:05<7:13:13,  2.95s/it]

 11%|█    

KeyboardInterrupt: 

In [93]:
for row in range(0,word2vec_features.index.size) :
    print row

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294


4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4172
4173
4174
4175
4176
4177
4178
4179
4180
4181
4182
4183
4184
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4201
4202
4203
4204
4205
4206
4207
4208
4209
4210
4211
4212
4213
4214
4215
4216
4217
4218
4219
4220
4221
4222
4223
4224
4225
4226
4227
4228
4229
4230
4231
4232
4233
4234
4235
4236
4237
4238
4239
4240
4241
4242
4243
4244
4245
4246
4247
4248
4249
4250
4251
4252
4253
4254
4255
4256
4257
4258
4259
4260
4261
4262
4263
4264
4265


6077
6078
6079
6080
6081
6082
6083
6084
6085
6086
6087
6088
6089
6090
6091
6092
6093
6094
6095
6096
6097
6098
6099
6100
6101
6102
6103
6104
6105
6106
6107
6108
6109
6110
6111
6112
6113
6114
6115
6116
6117
6118
6119
6120
6121
6122
6123
6124
6125
6126
6127
6128
6129
6130
6131
6132
6133
6134
6135
6136
6137
6138
6139
6140
6141
6142
6143
6144
6145
6146
6147
6148
6149
6150
6151
6152
6153
6154
6155
6156
6157
6158
6159
6160
6161
6162
6163
6164
6165
6166
6167
6168
6169
6170
6171
6172
6173
6174
6175
6176
6177
6178
6179
6180
6181
6182
6183
6184
6185
6186
6187
6188
6189
6190
6191
6192
6193
6194
6195
6196
6197
6198
6199
6200
6201
6202
6203
6204
6205
6206
6207
6208
6209
6210
6211
6212
6213
6214
6215
6216
6217
6218
6219
6220
6221
6222
6223
6224
6225
6226
6227
6228
6229
6230
6231
6232
6233
6234
6235
6236
6237
6238
6239
6240
6241
6242
6243
6244
6245
6246
6247
6248
6249
6250
6251
6252
6253
6254
6255
6256
6257
6258
6259
6260
6261
6262
6263
6264
6265
6266
6267
6268
6269
6270
6271
6272
6273
6274
6275
6276


8787
8788
8789
8790
8791
8792
8793
8794
8795
8796
8797
8798
8799
8800
8801
8802
8803
8804
8805
8806
8807
8808
8809
8810
8811
8812
8813
8814
8815
8816
8817
8818
8819
8820
8821
8822
8823
8824
8825
8826
8827
8828
8829
8830
8831
8832
8833
8834
8835
8836
8837
8838
8839
8840
8841
8842
8843
8844
8845
8846
8847
8848
8849
8850
8851
8852
8853
8854
8855
8856
8857
8858
8859
8860
8861
8862
8863
8864
8865
8866
8867
8868
8869
8870
8871
8872
8873
8874
8875
8876
8877
8878
8879
8880
8881
8882
8883
8884
8885
8886
8887
8888
8889
8890
8891
8892
8893
8894
8895
8896
8897
8898
8899
8900
8901
8902
8903
8904
8905
8906
8907
8908
8909
8910
8911
8912
8913
8914
8915
8916
8917
8918
8919
8920
8921
8922
8923
8924
8925
8926
8927
8928
8929
8930
8931
8932
8933
8934
8935
8936
8937
8938
8939
8940
8941
8942
8943
8944
8945
8946
8947
8948
8949
8950
8951
8952
8953
8954
8955
8956
8957
8958
8959
8960
8961
8962
8963
8964
8965
8966
8967
8968
8969
8970
8971
8972
8973
8974
8975
8976
8977
8978
8979
8980
8981
8982
8983
8984
8985
8986


In [127]:
db = pd.read_csv("/home/umang/Desktop/Bag_of_words.csv")
db.head()
print(db.shape)

(9910, 81)


In [120]:
opening_tags = ['op_tag','oi_tag','opc_tag','ow_tag','oa_tag','ocg_tag','odf_tag']
closing_tags = ['cac_tag','cap_tag','ccd_tag','cdf_tag','cff_tag','cth_tag','cw_tag']
all_opening = pd.DataFrame()
all_opening = db[db['Tag_name'].isin(opening_tags)]
all_closing = pd.DataFrame()
all_closing = db[db['Tag_name'].isin(closing_tags)] 



In [158]:
array = all_closing.values
X = array[:,7:81]
y = array[:,6]
for row in X:
    print row

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

In [159]:

num_feats = [74]
models = ['log_reg' , 'svm']
model_list = []
feat_list = []
accuracy_mean_list = []
accuracy_stdev_list = []
report_lr_list = []
for model in models:
    for feat_num in num_feats:
        model_list.append(model)
        feat_list.append(feat_num)
        accuracy_lr = []
        report_lr = []
        print model, feat_num
        for i in tqdm(range(0,20)):
            X_train, X_test, y_train,y_test = cross_validation.train_test_split(X, y, test_size = 0.30)
            
            selectkBest=SelectKBest(f_classif, k=feat_num)
            X_new_train=selectkBest.fit_transform(X_train, y_train)
            X_new_test=selectkBest.transform(X_test)
            #count_vectorizer = CountVectorizer()
            #X_train=count_vectorizer.fit_transform(X_train)
            #print "Vocabulary:", count_vectorizer.vocabulary
            if model == 'log_reg':
                model_used = LogisticRegression()
            if model == 'svm':
                model_used = svm.SVC()
            model_used.fit(X_new_train, y_train)
            predictions = model_used.predict(X_new_test)
            accuracy_lr.append(accuracy_score(y_test, predictions))
            y_pred = predictions
            y_true = y_test
            report_lr.append(precision_recall_fscore_support(y_true, y_pred, average= None , labels=closing_tags))
        report_lr = np.mean(report_lr,axis=0)
        precision_ = list(report_lr[0])
        precision_.append('precision')
        recall_ = list(report_lr[1])
        recall_.append('recall')
        f1_score_ = list(report_lr[2])
        f1_score_.append('f1_score')
        model_type = [model, feat_num]
        precision_.extend(model_type)
        recall_.extend(model_type)
        f1_score_.extend(model_type)
        report_lr_list.append(precision_)
        report_lr_list.append(recall_)
        report_lr_list.append(f1_score_)
        accuracy_mean_list.append(np.mean(accuracy_lr))
        accuracy_stdev_list.append(np.std(accuracy_lr))





  0%|          | 0/20 [00:00<?, ?it/s]



  5%|▌         | 1/20 [00:00<00:02,  8.93it/s]

log_reg 74






 10%|█         | 2/20 [00:00<00:02,  6.94it/s]



 15%|█▌        | 3/20 [00:00<00:02,  6.77it/s]



 20%|██        | 4/20 [00:00<00:02,  7.13it/s]



 25%|██▌       | 5/20 [00:00<00:02,  7.05it/s]



 30%|███       | 6/20 [00:00<00:02,  6.72it/s]



 35%|███▌      | 7/20 [00:01<00:01,  6.62it/s]



 40%|████      | 8/20 [00:01<00:01,  6.77it/s]



 45%|████▌     | 9/20 [00:01<00:01,  6.76it/s]



 50%|█████     | 10/20 [00:01<00:01,  6.74it/s]



 55%|█████▌    | 11/20 [00:01<00:01,  6.73it/s]



 60%|██████    | 12/20 [00:01<00:01,  6.72it/s]



 65%|██████▌   | 13/20 [00:01<00:01,  6.71it/s]/home/umang/.local/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [36 39 44 55 61 65 66 72] are constant.
  UserWarning)




 70%|███████   | 14/20 [00:02<00:00,  6.67it/s]



 75%|███████▌  | 15/20 [00:02<00:00,  6.77it/s]



 80%|████████  | 16/20 [00:02<00:00,  6.76it/s]



 85%|████████▌ | 17/20 [00:02<00:00,  6.75it/s]



 90%|████

svm 74






  5%|▌         | 1/20 [00:03<01:00,  3.17s/it]



 10%|█         | 2/20 [00:06<00:55,  3.06s/it]



 15%|█▌        | 3/20 [00:09<00:51,  3.03s/it]



 20%|██        | 4/20 [00:12<00:48,  3.02s/it]



 25%|██▌       | 5/20 [00:15<00:45,  3.01s/it]



 30%|███       | 6/20 [00:18<00:42,  3.00s/it]



 35%|███▌      | 7/20 [00:20<00:38,  3.00s/it]



 40%|████      | 8/20 [00:23<00:35,  2.99s/it]



 45%|████▌     | 9/20 [00:27<00:33,  3.01s/it]



 50%|█████     | 10/20 [00:30<00:30,  3.01s/it]



 55%|█████▌    | 11/20 [00:33<00:27,  3.01s/it]



 60%|██████    | 12/20 [00:36<00:24,  3.01s/it]



 65%|██████▌   | 13/20 [00:39<00:21,  3.01s/it]



 70%|███████   | 14/20 [00:42<00:18,  3.01s/it]



 75%|███████▌  | 15/20 [00:45<00:15,  3.01s/it]



 80%|████████  | 16/20 [00:48<00:12,  3.00s/it]



 85%|████████▌ | 17/20 [00:51<00:09,  3.00s/it]



 90%|█████████ | 18/20 [00:54<00:06,  3.00s/it]



 95%|█████████▌| 19/20 [00:57<00:03,  3.01s/it]



100%|██████████| 20/20 [01:00<00:00,

In [161]:
new_df = pd.DataFrame(report_lr_list,columns=closing_tags+['metric','model','features'])
print new_df
new_df.to_csv('/home/umang/Desktop/AM_Email_Tagging_Model_Building_Results/Bag_of_Words/Closing_Tags/classification.csv')

    cac_tag   cap_tag   ccd_tag   cdf_tag   cff_tag   cth_tag    cw_tag  \
0  0.571684  0.945315  0.652832  0.469679  0.605135  0.928564  0.742103   
1  0.686350  0.776500  0.323629  0.630774  0.499606  0.917134  0.329869   
2  0.623320  0.851887  0.431149  0.537709  0.546841  0.922644  0.452750   
3  0.571194  0.942246  0.679417  0.394362  0.630071  0.929851  0.000000   
4  0.617571  0.595680  0.239338  0.699933  0.419028  0.913864  0.000000   
5  0.593019  0.728469  0.352470  0.504146  0.502826  0.921633  0.000000   

      metric    model  features  
0  precision  log_reg        74  
1     recall  log_reg        74  
2   f1_score  log_reg        74  
3  precision      svm        74  
4     recall      svm        74  
5   f1_score      svm        74  


In [162]:
df = pd.DataFrame()
df['model'] = model_list
df['features'] = feat_list
df['accuracy_mean'] = accuracy_mean_list
df['accuracy_stdev'] = accuracy_stdev_list
df.to_csv('/home/umang/Desktop/AM_Email_Tagging_Model_Building_Results/Bag_of_Words/Closing_Tags/accuracy.csv')
df

,model,features,accuracy_mean,accuracy_stdev
0,log_reg,74,0.630837,0.008936
1,svm,74,0.587083,0.011160
